# Repo Link
https://github.com/mr-kelsey/fa25-aai520-group6.git

# Objective
Build an autonomous Investment Research Agent that:
1. Plans its research steps for a given stock symbol.
2. Uses tools dynamically (APIs, datasets, retrieval).
3. Self-reflects to assess the quality of its output.
4. Learns across runs (e.g., keeps brief memories or notes to improve future analyses).

# Code

In [ ]:
"""Imports"""

## Prompt Chaining:
### Ingest News

### Preprocess

### Classify

### Extract

### Summarize

## Routing:

In [ ]:
"""Direct content to the right specialist (e.g., earnings, news, or market analyzers)."""

## Evaluator–Optimizer:
### Generate analysis

### Evaluate quality

### Refine using feedback

# Summary

* Agent Design and Workflows
* Agent Functions and Capabilities
* Evaluation and Iteration